In [22]:
import pandas as pd
import json
import requests

In [23]:
# URL of the JSON data
url = "https://ollama-models.zwz.workers.dev/"

# Send a GET request to the URL
response = requests.get(url)

# Load the JSON data from the response
data = json.loads(response.text)

data["models"]
# convert it to pandas df
df = pd.DataFrame(data["models"])
# explode tags
df = df.explode("tags")
df = df[["name", "tags"]]

In [25]:
import re


def extract_params(tag):
    match = re.search(r"^(\d+\.\d+)|(\d+x\d+b)|(\d+b)", tag)
    if match:
        return match.group()
    else:
        return None


df["nparams"] = df["tags"].apply(extract_params)
# remove rows with nparams == None
df = df[df["nparams"].notna()]
# if any nparams doesn't end with letter b, append it
df["nparams"] = df["nparams"].apply(lambda x: x + "b" if x[-1] != "b" else x)

In [27]:
def extract_quant(tag):
    # Match either 'q' followed by alphanumeric characters or 'fp' followed by alphanumeric characters
    match = re.search(r"(q|fp)\w+", tag)
    if match:
        return match.group()
    else:
        return None


df["quant"] = df["tags"].apply(extract_quant)
df = df[df["quant"].notna()]
# capitalize the columnn
df["quant"] = df["quant"].apply(lambda x: x.upper())

In [28]:
df = df[["name", "nparams", "quant"]]

In [29]:
# remove any duplicate rows
df = df.drop_duplicates()

In [30]:
# Convert the DataFrame to a dictionary
df_dict = (
    df.set_index(["name", "nparams"])["quant"]
    .groupby(["name", "nparams"])
    .apply(list)
    .to_dict()
)

# Create the desired hierarchical structure
result = {}
for (name, nparams), quant in df_dict.items():
    if name not in result:
        result[name] = {}
    result[name][nparams] = quant

# Save the hierarchical structure to a JSON file
with open("ollama_models.json", "w") as f:
    json.dump(result, f, indent=4)